# Homework 3: Orchestration

**Downloading the data "Yellow Taxi Trip Records" data for March 2023, from https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page**

In [1]:
!mkdir -p data
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet -P ./data

--2025-06-03 22:52:47--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 65.8.245.51, 65.8.245.50, 65.8.245.178, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|65.8.245.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56127762 (54M) [binary/octet-stream]
Saving to: ‘./data/yellow_tripdata_2023-03.parquet.1’

yellow_tripdata_202 100%[===================>]  53,53M  3,24MB/s    in 15s     

2025-06-03 22:53:03 (3,52 MB/s) - ‘./data/yellow_tripdata_2023-03.parquet.1’ saved [56127762/56127762]



**Import all necessary modules**

In [2]:
import pandas as pd

**Read March data**

In [3]:
df = pd.read_parquet('./data/yellow_tripdata_2023-03.parquet')

**Retrieve number of rows and columns**

In [4]:
df.shape

(3403766, 19)

In [5]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [6]:
df = read_dataframe('./data/yellow_tripdata_2023-03.parquet')
df.shape

(3316216, 20)

Run mlflow local server in a terminal with:  
```
mlflow ui --backend-store-uri sqlite:///mlflow.db  --default-artifact-root ./artifacts
```

Run prefect local server in a terminal with:  
```
prefect server start
```

In [ ]:
def register_model(data_path: str, top_n: int):

    client = MlflowClient()

    # Retrieve the top_n model runs and log the models
    experiment = client.get_experiment_by_name(HPO_EXPERIMENT_NAME)
    runs = client.search_runs(
        experiment_ids=experiment.experiment_id,
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=top_n,
        order_by=["metrics.rmse ASC"]
    )
    for run in runs:
        train_and_log_model(data_path=data_path, params=run.data.params)

    # Select the model with the lowest test RMSE
    experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
    # best_run = client.search_runs( ...  )[0]
    best_run = client.search_runs(
        experiment_ids=experiment.experiment_id,
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=1,
        order_by=["metrics.rmse ASC"]
    )[0]

    # Register the best model
    # mlflow.register_model( ... )
    run_id = best_run.info.run_id
    mlflow.register_model(
        model_uri=f"runs:/{run_id}/models",
        name='best-model-final2-realone'
    )

In [ ]:
import mlflow
from mlflow.tracking import MlflowClient

mlflow.set_tracking_uri('http://localhost:5000')
mlflow.set_experiment('nyc-taxi-experiment')
client = MlflowClient()
experiment = client.get_experiment_by_name('nyc-taxi-experiment')
runs = client.search_runs(
    experiment_ids=experiment.experiment_id,
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=top_n,
    order_by=["metrics.rmse ASC"]
)
